# Chaper.19 GAN

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Reshape,Conv2D,Dropout,Flatten,BatchNormalization,Activation,Input,UpSampling2D,LeakyReLU
from tensorflow.keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
generator = Sequential()
generator.add(Dense(7*7*128, input_dim=100, activation=LeakyReLU(.2)))
generator.add(BatchNormalization())
generator.add(Reshape(7, 7, 128))
generator.add(UpSampling2D())

